In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, Markdown

def compound(principal, interest_rate, number_of_compounds_yearly, years):
    return principal*(1+interest_rate/number_of_compounds_yearly)**(years*number_of_compounds_yearly)

def reverse_compound(amount, interest_rate, years):
    return amount*(interest_rate+1)**(-years)




def house_calculations(house_cost,deposit,interest_rate,repayment_term,monthly_rent,roi,
                       annual_growth_house,contribution,inflation,pay_raise):
    starting_principal = house_cost - deposit
    
    annual_growth_house = annual_growth_house / 100
    roi = roi / 100
    interest_rate = interest_rate / 100
    inflation = inflation / 100
    pay_scale = pay_raise / 100
    
    if deposit > house_cost:
        display(Markdown("###### Invest in whatever has the higher rate of return. You can afford it."))
        return
    
    starting_monthly_contribution = contribution
    
    suggested_income = (starting_monthly_contribution*12)/(0.33)
    display(Markdown("---"))
    display(Markdown("###### Starting yearly contribution: ${:,}".format(int(starting_monthly_contribution*12))))
    display(Markdown("###### Suggested yearly income: ${:,}".format(int(suggested_income))))



    p = starting_principal

    interests = []
    contributions = []
    ps = []
    for year in range(repayment_term):
        p = compound(p,interest_rate,4,1)
        interest = p*interest_rate
        contribution = compound(contribution, pay_scale,1,1)
        remaining = (contribution*12)-interest
        #print(interest)
        #print(contribution*12)
        if remaining < 0:
            print("Can't afford interest!")
            
        p = p - remaining
        if p < 0:
            p = 0
            interest = 0
            break
        interests.append(interest)
        contributions.append(contribution)
        ps.append(p)
        #print(p,interest)

    display(Markdown("###### Total interest paid: \${:,} which is \${:,} per month".format(int(sum(interests)),int(sum(interests)/(len(ps)*12)))))
    display(Markdown("###### Total contributions made: \${:,}".format(int(sum(contributions*12)))))


    rent = monthly_rent*12
    rents = []
    investments = []
    bank_balance = deposit

    for i in range(len(ps)):
        rents.append(compound(rent,inflation,1,1))
        bank_balance = compound(bank_balance,roi,1,1)
        bank_balance += contributions[i]*12 - rents[i]

    inf_adjusted_ret = int(reverse_compound(bank_balance,inflation,len(ps)))
    inf_adjusted_cont = reverse_compound(sum([c * 12 for c in contributions]),inflation,len(ps))
    percent_increase_real = int(inf_adjusted_ret/inf_adjusted_cont*100)-100
    print("")
    display(Markdown("###### Net worth if rented (assuming rent inflation): \${:,}".format(int(bank_balance))))
    display(Markdown("###### Equivalent to ~\${:,} today and {:,}% return in real terms".format(
        inf_adjusted_ret,percent_increase_real)))


    house_value = compound(house_cost,annual_growth_house,1,len(ps))
    display(Markdown("###### Net worth if bought house: ${:,}".format(int(house_value))))
    house_value_today = reverse_compound(house_value,inflation,len(ps))
    percent_increase_house_real = int(house_value_today/inf_adjusted_cont*100-100)
    display(Markdown("###### Equivalent to ~\${:,} today and {:,}% return in real terms".format(
        int(house_value_today),percent_increase_house_real)))

    N = len(ps)
    ind = np.arange(N)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence


    fig = plt.figure(figsize=(16, 16), dpi=100, facecolor='w')
    p1 = plt.bar(ind, ps, width, edgecolor='black',linewidth=1)
    p2 = plt.bar(ind, interests, width,
                 bottom=ps, edgecolor='black',linewidth=1)

    tots = list(map(lambda x,y,z: x+y-z*12 if x+y-z*12 > 0 else 0,ps,interests,contributions))
    tops = list(map(lambda x,y,z: x+y-z if x+y-z > 0 else 0,ps,interests,tots))
    p3 = plt.bar(ind, tops, width,
                bottom=tots,fill=False, hatch="/")

    plt.ylabel('Amount in AUD')
    plt.xlabel('Year')
    plt.title('Loan Payments Over {} Years'.format(N))
    plt.xticks(np.arange(0,len(ps)),np.arange(1,len(ps)+1))
    yticks = list(np.arange(0, 400000, 50000))
    yticks.append(interests[0]+ps[0])
    #plt.yticks(yticks)
    plt.legend((p1, p2,p3), ('Principal', 'Interest','Yearly Contribution'))

    plt.show()

## Maybe make a plot showing net worth?
## Add a check for debt
style = {'description_width': 'initial', 'max-width':'100%'}
iplot = interactive(house_calculations, 
         house_cost = widgets.IntSlider(value=400000,min=100000,max=2000000, step=10000,
    description='House Cost', style=style),
         deposit = widgets.IntSlider(value=140000,min=50000,max=1000000, step=10000,
    description='Deposit', style=style),
         interest_rate = widgets.FloatSlider(value=4.60,min=2,max=10.0, step=0.01,
    description='Interest Rate', style=style),
         repayment_term = widgets.IntSlider(value=15,min=5,max=30, step=1,
    description='Repayment Term', style=style),
         monthly_rent = widgets.IntSlider(value=2000,min=500,max=5000, step=100,
    description='Rent', style=style),
         roi = widgets.FloatSlider(value=7.5,min=0,max=15.0, step=0.1,
    description='Investment Growth Rate', continuous_update=False, style=style),
         annual_growth_house = widgets.FloatSlider(value=11.0,min=0,max=15.0, step=0.1,
    description='Annual House Growth', style=style),
        contribution = widgets.IntSlider(value=3000,min=500,max=5000, step=100,
    description='Monthly Mortgage', style=style),
        inflation = widgets.FloatSlider(value=2,min=0,max=10.0, step=0.1,
    description='Inflation', style=style),
        pay_raise = widgets.FloatSlider(value=3,min=0,max=10, step=0.1,
    description='Yearly Pay Increase', style=style)
        )

iplot

interactive(children=(IntSlider(value=400000, description='House Cost', max=2000000, min=100000, step=10000, style=SliderStyle(description_width='initial')), IntSlider(value=140000, description='Deposit', max=1000000, min=50000, step=10000, style=SliderStyle(description_width='initial')), FloatSlider(value=4.6, description='Interest Rate', max=10.0, min=2.0, step=0.01, style=SliderStyle(description_width='initial')), IntSlider(value=15, description='Repayment Term', max=30, min=5, style=SliderStyle(description_width='initial')), IntSlider(value=2000, description='Rent', max=5000, min=500, step=100, style=SliderStyle(description_width='initial')), FloatSlider(value=7.5, continuous_update=False, description='Investment Growth Rate', max=15.0, style=SliderStyle(description_width='initial')), FloatSlider(value=11.0, description='Annual House Growth', max=15.0, style=SliderStyle(description_width='initial')), IntSlider(value=3000, description='Monthly Mortgage', max=5000, min=500, step=100, style=SliderStyle(description_width='initial')), FloatSlider(value=2.0, description='Inflation', max=10.0, style=SliderStyle(description_width='initial')), FloatSlider(value=3.0, description='Yearly Pay Increase', max=10.0, style=SliderStyle(description_width='initial')), Output()), _dom_classes=('widget-interact',))